In [2]:
import numpy as np 
import torch
import os 
import re
import matplotlib.pyplot as plt
import spectrogram
#import torchaudio 


In [3]:
from scipy.io import wavfile
data_path = 'data/'
## Only Diffusiing from a single sample (TODO: Load many samples)
sample_name = os.listdir(data_path)[2]
# Specify the path to the .wav file
wav_file = os.path.join(data_path, sample_name)

# Load the .wav file as a numpy array
sample_rate, audio_data = wavfile.read(wav_file)
# Avg together Left and Right channels to get Mono Signal
audio_data = np.mean(audio_data, axis=1)
## Transform to torch tensor
audio_data = torch.from_numpy(audio_data).float().unsqueeze(0)

/tmp/ipykernel_3877/1907631780.py:9: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio_data = wavfile.read(wav_file)


In [4]:
## Import Audio for playback
from IPython.display import Audio
## TODO: Figure out why certain combinations of n_mels cause the conversion to fail
spectrogram_transforms = spectrogram.Spectrogram(
    n_fft=1024,
    win_length=1024,
    hop_length=512,
    n_mels=80,
    sample_rate=sample_rate,
)

## TODO: Add in VAE from audioldm

mel_scale = spectrogram_transforms.mel_transform(audio_data)
#reconstructed_audio = spectrogram_transforms.inverse_transform(mel_scale)

## Split the mel_scale data from 

#Audio(torch.cat((audio_data, reconstructed_audio), dim=1), rate=sample_rate)

In [5]:
## Reshape from 1x80xn to (n//1470)x80x1470
# Calculate the desired size after reshaping
N = mel_scale.size(2)
new_size = (N // 1480, 1480, 80)
remaining = N % 1480
## cut the remaining part
cut_mel_scale = mel_scale[:, :, :-remaining]
# Reshape the padded tensor
reshaped_mel_scale = cut_mel_scale.swapdims(1,2).reshape(new_size).swapdims(1,2)

In [6]:
## create x axis based off sample rate
time = np.arange(0, len(audio_data), 1) / sample_rate
plt.plot(time, audio_data)

In [ ]:
# ## Import DDPM and model for processing data
# from Models.models import Mel_Convolv
# from DDPM import DDPM_Scheduler
# import torch

# model = Mel_Convolv(80, 1470)
# test = torch.rand(1, 1, 80, 1470)
# model(test)

tensor([[[[-0.0248,  0.0207,  0.0211,  ...,  0.0391, -0.0187,  0.0258],
          [ 0.0249,  0.0490,  0.0712,  ...,  0.0537,  0.0663,  0.0036],
          [ 0.0567,  0.0305,  0.0260,  ...,  0.0430,  0.0373,  0.0505],
          ...,
          [ 0.0645,  0.0197,  0.0099,  ...,  0.0265,  0.1017,  0.0488],
          [ 0.0041,  0.0016,  0.0313,  ...,  0.0328,  0.0424, -0.0242],
          [ 0.0232,  0.0281,  0.0456,  ..., -0.0173,  0.0023,  0.0475]]]],
       grad_fn=<ConvolutionBackward0>)